Goal : Add the modulo colour code. Preprocessing of grids. treat a larger set of tasks. 

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from pathlib import Path
from collections import defaultdict
from itertools import product
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations,permutations
from sklearn.tree import *
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
import random
from math import floor

In [2]:
data_path = Path("/kaggle/input/abstraction-and-reasoning-challenge")
train_path = data_path/'training'
test_path = data_path/'test'
evaluation_path = data_path/'evaluation'

In [3]:
import json
import os
from os.path import join as path_join


def load_data(path):
    tasks = pd.Series()
    for file_path in os.listdir(path):
        task_file = path_join(path, file_path)

        with open(task_file, 'r') as f:
            task = json.load(f)

        tasks[file_path[:-5]] = task
    return tasks

In [4]:
train_tasks = load_data('../input/abstraction-and-reasoning-challenge/training/')
evaluation_tasks = load_data('../input/abstraction-and-reasoning-challenge/evaluation/')
test_tasks = load_data('../input/abstraction-and-reasoning-challenge/test/')
train_tasks.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


b91ae062    {'train': [{'input': [[6, 7, 0], [0, 6, 6], [0...
b27ca6d3    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
7f4411dc    {'train': [{'input': [[0, 7, 0, 0, 0, 0, 0, 0,...
9d9215db    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
11852cab    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
dtype: object

In [5]:
## Plotting functions

import matplotlib.pyplot as plt
from matplotlib import colors

def plot_task(task,j=0,k=0):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 4, figsize=(15,15))
    axs[0].imshow(task['train'][j]['input'], cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Train Input')
    axs[1].imshow(task['train'][j]['output'], cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Train Output')
    axs[2].imshow(task['test'][k]['input'], cmap=cmap, norm=norm)
    axs[2].axis('off')
    axs[2].set_title('Test Input')
    axs[3].imshow(task['test'][k]['output'], cmap=cmap, norm=norm)
    axs[3].axis('off')
    axs[3].set_title('Test Output')
    plt.tight_layout()
    plt.show()

In [6]:
def plot_images(images):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    m = len(images)
    fig, axs = plt.subplots(1, m, figsize=(15,15))
    for i in range(m):
        axs[i].imshow(images[i], cmap=cmap, norm=norm)
        axs[i].axis('off')
        axs[i].set_title('Image' + str(i))
    plt.tight_layout()
    plt.show()

DSL

In [7]:
class fund_unit:
    
    def __init__(self,colour,position,ambient = -1):
        self.colour = colour
        self.position = position #list of places
        l1 = [x[0] for x in position]
        l2 = [x[1] for x in position]
        a = min(l1)
        corners = get_corners(position)
        self.top_right = corners['top_right']
        self.top_left = corners['top_left']
        self.bottom_left = corners['bottom_left']
        self.bottom_right = corners['bottom_right']
        self.colour = colour
        self.ambient = ambient
        self.right = corners['right']
        self.left = corners['left']
        self.bottom = corners['bottom']
        self.top = corners['top']

In [8]:
def create(colour,position):
    unit = fund_unit(colour,position)
    return(unit)

## Attribute functions

In [9]:
#units is a list
def get_colour_dict(units):
    colours = {}
    for j in range(10):
        colours[j] = []
        for unit in units:
            if unit.colour == j:
                colours[j].append(unit)
        colours[j].sort(key = lambda unit: (unit.top_left[0],unit.top_left[1]))
    return(colours)    

In [10]:
def get_corners(position):
    l1 = [x[0] for x in position]
    l2 = [x[1] for x in position]
    a = min(l1)
    l3 = []
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    b = min(l3)
    c = max(l3)
    corners = {'top_left' : np.array([a,b]),'top_right' : np.array([a,c])}
    a = max(l1)
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    b = min(l3)
    c = max(l3)
    corners['bottom_left'] = np.array([a,b])
    corners['bottom_right'] = np.array([a,c])
    # get the coordinates of the top cell
    a = min(l1)
    l3 = []
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['top'] = np.array([a,l3[m]])
    #coordinates of the bottom cell
    a = max(l1)
    l3 = []
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['bottom'] = np.array([a,l3[m]])    
    #coordinates of the left cell
    a = min(l2)
    l3 = []
    for x in position:
        if x[1] == a:
            l3.append(x[0])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['left'] = np.array([l3[m],a])
    #coordinates of the right cell 
    a = max(l2)
    l3 = []
    for x in position:
        if x[1] == a:
            l3.append(x[0])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['right'] = np.array([l3[m],a])
    return(corners)

In [11]:
def get_size_list(units):
    units.sort(key = lambda u : len(u.position))
    return(units)    

In [12]:
# returns chunks of unit1 which are copies of unit2 after moving it. 
def pattern_matching(unit1,unit2,with_colour = False):
    if with_colour and not unit1.colour == unit2.colour:
        return([])
    else:
        l = []
        matches = []
        y = unit2.position[0]
        for x in unit1.position:
            l.append(x-y)
        s1 = [(a[0],a[1]) for a in unit1.position]        
        s1 = set(s1)
        for z in l:
            p2 = [y + z for y in unit2.position]
            p2 = [(a[0],a[1]) for a in p2]
            p2 = set(p2)
            if p2.issubset(s1):
                p2 = list(p2)
                p2 = [np.array([a[0],a[1]]) for a in p2]
                matches.append([p2,z])
        matches.sort(key = lambda a : abs(a[1][0]) + abs(a[1][1]))
        return(matches)                

In [13]:
# sorts a list of units by size
def sort_by_size(l):
    l.sort(key = lambda u : len(u.position))
    return(l)

In [14]:
def sort_by_position(l,base = 0):
    if base != 3 and base !=7:
        l.sort(key = lambda u : (give_base(u,base)[0],give_base(u,base)[1]))
    if base == 7:
        l.sort(key = lambda u : (give_base(u,base)[1],give_base(u,base)[0]))
    if base == 3:
        l.sort(key = lambda u : (-give_base(u,base)[1],give_base(u,base)[0]))
    return(l)

def give_base(unit,base):
    d = {}
    d[0] = unit.top_left
    d[1] = unit.top
    d[2] = unit.top_right
    d[3] = unit.right
    d[4] = unit.bottom_right
    d[5] = unit.bottom 
    d[6] = unit.bottom_left
    d[7] = unit.left
    return(d[base])

## Change unit in place / Get units 

In [15]:
def change_colour(l,c):
    for unit in l:
        unit.colour = c
    return(l)

In [16]:
def get_units(img,with_diag = False):
    img = np.array(img)
    r,c = img.shape
    ambient = img.shape
    E = {}
    V = []
    for i in range(r):
        for j in range(c):
            V.append((i,j))
            E[(i,j)] = [] 
    for i in range(r):
        for j in range(c):
            if i-1 >= 0 and img[i-1,j] == img[i,j]:
                    E[(i,j)].append((i-1,j))
            if i+1 <= r-1 and img[i+1,j] == img[i,j]:
                E[(i,j)].append((i+1,j))
            if j-1 >= 0 and img[i,j-1] == img[i,j]:
                E[(i,j)].append((i,j-1))
            if j+1 <= c-1 and img[i,j+1] == img[i,j]:
                E[(i,j)].append((i,j+1))
            if with_diag:    
                if i-1 >= 0 and j-1 >= 0 and img[i-1,j-1] == img[i,j]:
                    E[(i,j)].append((i-1,j-1))
                if i-1 >= 0 and j+1 <= c-1 and img[i-1,j+1] == img[i,j]:
                    E[(i,j)].append((i-1,j+1))
                if i+1 <= r-1 and j-1 >= 0 and img[i+1,j-1] == img[i,j]:
                    E[(i,j)].append((i+1,j-1))
                if i+1 <= r-1 and j+1 <= c-1 and img[i+1,j+1] == img[i,j]:
                    E[(i,j)].append((i+1,j+1))
    parent = DFS(V,E)
    units = []
    for v in parent.keys():
        if len(parent[v]) > 0:
            temp = [np.array([x[0],x[1]]) for x in parent[v]]
            unit = fund_unit(img[v],temp,ambient)
            units.append(unit)
    units.sort(key = lambda unit: (unit.top_left[0],unit.top_left[1]))        
    return(units)

In [17]:
def offset(coordinates,a,b):
    new_x = coordinates[0] + a
    new_y = coordinates[1] + b
    return(np.array([new_x,new_y]))

In [18]:
def DFS(V,E):
    colour = {}
    parent = {}
    for v in V:
        colour[v] = 0
        parent[v] = []
    for v in V:
        if colour[v] == 0:
            DFS_search(v,E,parent[v],colour)
    return(parent)        

In [19]:
def DFS_search(v,E,l,colour):
    l.append(v)
    colour[v] = 1
    for e in E[v]:
        if colour[e] == 0:
            DFS_search(e,E,l,colour)
    colour[v] = 2    

In [20]:
# f returns a list. We want it to act on every element of a list
def do_for_list(l,f):
    t = []
    for x in l:
        t = t + f(x)
    return(t)

## Check for contractibility and blank space

In [21]:
def get_genus(unit,with_diag = False):
    r,c = unit.ambient
    l = boundary_img(r,c)
    if check_belongs_to(l,unit.position):
        is_a_boundary = True
    else:
        is_a_boundary = False
    #i = 0
    #count = 0
    #while check_belongs_to([l[i]],unit.position):
    #    count += 1
    #    i += 1
    #    if i > len(l)-1:
    #        break
    #if count < 2*c + 2*r - 4:
    #    start_key = l[i]
    #    is_a_boundary = False
    #else:
    #    is_a_boundary = True
    u = fund_unit(1,unit.position,unit.ambient)
    img = concat([u],u.ambient)
    units = get_units(img,with_diag)
    enclosed_area = []
    g = 0 
    for v in units:
        if check_empty_intersection(l,v.position) and check_empty_intersection(v.position,unit.position):
            g += 1
    return(g)  

In [22]:
def check_contractible(unit,with_diag = False):
    g = get_genus(unit,with_diag)
    if g == 0:
        return(True)
    else:
        return(False)

In [23]:
def get_non_contractible(units,with_diag = False,with_background = False):
    non_contract = []
    units = sort_by_size(units)
    if not with_background and units[-1] == 0:
        units = units[:-1]
    for u in units:
        if not check_contractible(u,with_diag):
            non_contract.append(u)
    return(non_contract)   

In [24]:
def get_up_to_homotopy(units):
    t = []
    for u in units:
        if not check_contractible(u):
            t.append(u)
    return(t)        

## Check grid

In [25]:
def check_identical(l):
    u0 = l[0]
    t = 0
    for u in l:
        if len(u.position) == len(u0.position):
            p = pattern_matching(u,u0)
            if len(p) >= 1:
                t += 1
    if t == len(l):
        return(True)
    else:
        return(False)

In [26]:
def check_grid_new(img,get_diag = False):
    img = np.array(img)
    ambient = np.array(img).shape
    units = sort_by_size(get_units(img,get_diag))
    for u in units:
        c = 0
        v = []
        for w in units:
            if w != u:
                v.append(w)
        r = {}
        for w in v:
            x = w.top_left
            if x[0] in r.keys():
                r[x[0]].append(w)
            else:
                r[x[0]] = [w]
        temp = []
        for k in r.keys():
            temp.append(len(r[k]))
        if len(set(temp)) > 1:
            c = 1 
        for k in r.keys():
            if not check_identical(r[k]):
                c = 1
        if c == 0:
            return([u])
    return([]) 

In [27]:
def check_grid(img,get_diag = False):
    img = np.array(img)
    ambient = np.array(img).shape
    units = sort_by_size(get_units(img,get_diag))
    background = units[-1]
    u0 = units[0]
    c = 0
    for u in units[:-1]:
        if len(u0.position) == len(u.position):
            l = pattern_matching(u0,u)
            if len(l) > 0:
                c += 1
    if c == len(units)-1:
        return(True)
    else:
        return(False)    

In [28]:
# background is the unit of largest size

def get_background(img,get_diag = False):
    img = np.array(img)
    ambient = np.array(img).shape
    units = get_units(img,get_diag)
    background = sort_by_size(units)[-1]
    return(background)

## collecting units that are surrounded

In [29]:
def check_actual_coord(x,y,r,c):
    if x >= 0 and x <= r-1 and y >= 0 and y <= c-1:
        return(True)
    else:
        return(False)

In [30]:
def udlr(x,r,c):
    a,b = x[0],x[1]
    l = []
    x_up = [a-1,b]
    x_down = [a+1,b]
    x_right = [a,b+1]
    x_left = [a,b-1]
    t = [x_up,x_down,x_right,x_left]
    for i in range(4):
        if check_actual_coord(t[i][0],t[i][1],r,c):
            l.append(np.array(t[i]))
    return(l)       

In [31]:
def is_surrounded(u1,u2):
    r,c = u1.ambient
    val = True
    for x in u1.position:
        t = udlr(x,r,c)
        m = len(t)
        if m < 4:
            val = False
        for i in range(m):
            if not check_belongs_to([t[i]],u1.position):
                if not check_belongs_to([t[i]],u2.position):
                    val = False
    return(val)     

In [32]:
# Returns a dict - surrounded, where surrounded[u] is the list of units v surrounded by u.

def get_surrounded(units):
    surrounded = {}
    for u in units:
        surrounded[u] = []
        for v in units:
            if is_surrounded(v,u):
                surrounded[u].append(v)
    return(surrounded)    

In [33]:
def surrounded_list(units):
    surrounded = []
    d = get_surrounded(units)
    l = []
    for k in d.keys():
        l = l + d[k]
    return(l)    

## Get Shapes which are contained in other shapes

In [34]:
def boundary_img(r,c):
    l = []
    for j in range(c):
        l.append(np.array([0,j]))
    for j in range(r):
        l.append(np.array([j,c-1]))
    for j in range(c):
        l.append(np.array([r-1,j]))
    for j in range(r):
        l.append(np.array([j,0]))
    return(l)    

In [35]:
def get_intersection(l1,l2):
    l11 = [(x[0],x[1]) for x in l1]
    l22 = [(x[0],x[1]) for x in l2]
    l = list(set(l11)&set(l22))
    l = [np.array(x) for x in l]
    return(l)

In [36]:
def check_empty_intersection(l1,l2):
    l = get_intersection(l1,l2)
    if len(l) > 0:
        return(False)
    else:
        return(True)

In [37]:
# Assume the container is not black

def get_enclosed_area(unit):
    r,c = unit.ambient
    l = boundary_img(r,c)
    i = 0
    count = 0
    while check_belongs_to([l[i]],unit.position):
        count += 1
        i += 1
        if i > len(l)-1:
            break
    if count < 2*c + 2*r - 4:
        start_key = l[i]
        is_a_boundary = False
    else:
        is_a_boundary = True
    u = fund_unit(1,unit.position,unit.ambient)
    img = concat([u],u.ambient)
    units = get_units(img)
    enclosed_area = []
    if is_a_boundary:
        for v in units:
            if check_belongs_to([v.top_left],[u.top_left]):
                enclosed_area += v.position                
    if len(units) > 2 and not is_a_boundary:
        for v in units:
            if not check_belongs_to([v.top_left],[u.top_left]) and check_empty_intersection(l,v.position):
                enclosed_area += v.position
    return(enclosed_area)            

In [38]:
def contained_list(units):
    contained = []
    enclosed_area = []
    for u in units:
        if u.colour != 0:
            enclosed_area += get_enclosed_area(u)
    for v in units:
        if check_belongs_to(v.position,enclosed_area):
            contained.append(v)
    return(contained)                   

In [39]:
def contained_in(units):
    d = {}
    l = detect_regular_non_contractible(units)
    for u in l:
        d[u] = []
        x = u.top_left
        y = u.bottom_right
        l1 = list(range(x[0],y[0]+1))
        l2 = list(range(x[1],y[1]+1))
        temp = list(itertools.product(l1,l2))
        p = []
        for t in temp:
            p.append(np.array([t[0],t[1]]))
        pdiff = []    
        for v in p:
            r0 = 0
            for z in u.position:
                if v[0] == z[0] and v[1] == z[1]:
                    r0 += 1
            if r0 == 0:
                pdiff.append(v)
        udiff = fund_unit(0,pdiff,u.ambient)
        for v in units:
            if check_belongs_to(v.position,u.position):
                d[u].append(v)
    return(d)            

## Extracting simple shapes

In [40]:
def is_regular(unit):
    x = unit.top_left
    y = unit.bottom_right
    l1 = list(range(x[0],y[0]+1))
    l2 = list(range(x[1],y[1]+1))
    temp = list(itertools.product(l1,l2))
    p = []
    for t in temp:
        p.append(np.array([t[0],t[1]]))
    u0 = fund_unit(0,p,unit.ambient)
    if len(p) == len(unit.position):
        l = pattern_matching(u0,unit)
        if len(l) > 0:
            if len(l1) == len(l2):
                return('square')
            if len(l1) == 1 and len(l2) > len(l1):
                return('vertical line')
            if len(l2) == 1 and len(l1) > len(l2):
                return('horizontal line')
            if len(l1) > len(l2):
                return('horizontal rectangle')
            if len(l2) > len(l1):
                return('vertical rectangle')
    return('-1')        

In [41]:
def get_regular_shapes(units):
    d = {}
    d['square'] = []
    d['vertical line'] = []
    d['horizontal line'] = []
    d['horizontal rectangle'] = []
    d['vertical rectangle'] = []
    d['-1'] = []
    for u in units:
        d[is_regular(u)].append(u)
    return(d)   

In [42]:
# returns a dict whose keys are elements of the list 'units' and whose values are equivalence classes coded as integers or the list of equivalent shapes.  

def shape_equivalences(units,return_lists = True,remove_background = False): 
    #img = np.array(img)
    #ambient = np.array(img).shape
    units = sort_by_size(units)
    if remove_background:
        units = units[:-1]    
    d = {}
    d[units[0]] = []
    is_key = {}
    for u in units:
        is_key[u] = False
    for u in units:
        c = 0
        for v in units:
            if is_key[v]:
                if len(u.position) == len(v.position):
                    l = pattern_matching(u,v)
                    if len(l) > 0:
                        c += 1
                        d[v].append(u)
        if c == 0:
            d[u] = [u]
            is_key[u] = True
    m = len(d.keys())
    shape_equivalences = {}
    for j,key in enumerate(d.keys()):
        for u in d[key]:
            shape_equivalences[u] = j
    if return_lists:
        return(d)
    else:
        return(shape_equivalences)

In [43]:
def get_task_num(img,task):
    m = len(task['train'])
    for i in range(m):
        if np.all(img == task['train'][i]['input']):
            return(i)

In [44]:
# Returns list of units of an image which also occur in all other images of the training set. num = -1 when img is the test image

def common_shapes_across_images(img,task,img_units,num):
    #if num == -1:
    #    num = get_task_num(img,task)
    m = len(task['train'])
    all_units = []
    task_number = {}
    for i in range(m):
        if i != num:
            units = get_units(task['train'][i]['input'])
        else:
            units = img_units
        for u in units:
            task_number[u] = i
        all_units += units
    if num == -1:
        units = img_units
        all_units += img_units
        for u in units:
            task_number[u] = i
    d = shape_equivalences(all_units) 
    shapes_across_images = []
    for x in d.keys():
        l = [task_number[y] for y in d[x]]
        if len(set(l)) == m:
            shapes_across_images.append(x)
    final_list = []
    for x in shapes_across_images:
         for u in d[x]:
                if task_number[u] == num:
                    final_list.append(u)
    return(final_list)       

In [45]:
def shape_equivalences_counts(units,remove_background = False):
    d = {}
    s = shape_equivalences(units,True,remove_background)
    for x in s.keys():
        m = len(s[x])
        if m in d.keys():
            d[m] += s[x]
        else:
            d[m] = s[x]
    return(d)      

In [46]:
# checks if l1 is a subset of l2. Both are lists of np arrays of length 2.
def check_belongs_to(l1,l2):
    r = 0 
    for z in l1:
        for x in l2:
            if x[0] == z[0] and x[1] == z[1]:
                r += 1
    if r == len(l1):
        return(True)
    else:
        return(False)

In [47]:
def check_this_please(u,x1,x2):
    l1 = list(range(x1[0],x2[0]+1))
    l2 = []
    for t in l1:
        l2.append(np.array([t,x1[1]]))
    if check_belongs_to(l1,u.position):
        count = 1
    else:
        count = 0
    return(count)  

In [48]:
import itertools
# selects the regular non contractible elements of units i.e. those elements of the list units whose hole is a regular object. 

def detect_regular_non_contractible(units):
    #img = np.array(img)
    #ambient = np.array(img).shape
    units = sort_by_size(units)
    regular_non_contractible = []
    for u in units:
        x = u.top_left
        y = u.bottom_right
        l1 = list(range(x[0],y[0]+1))
        l2 = list(range(x[1],y[1]+1))
        temp = list(itertools.product(l1,l2))
        p = []
        for t in temp:
            p.append(np.array([t[0],t[1]]))
        pdiff = []    
        for v in p:
            r0 = 0
            for z in u.position:
                if v[0] == z[0] and v[1] == z[1]:
                    r0 += 1
            if r0 == 0:
                pdiff.append(v)
        if len(pdiff) > 0:        
            udiff = fund_unit(0,pdiff,u.ambient) 
            u0 = fund_unit(0,p,u.ambient)
            s = is_regular(udiff)
            if s != '-1':
                count = 0
                x1 = udiff.top_left
                x2 = udiff.top_right
                l1 = list(range(x1[1],x2[1]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([x1[0]-1,t]))
                if check_belongs_to(l2,u.position):
                    count += 1
                x1 = udiff.top_right
                x2 = udiff.bottom_right
                l1 = list(range(x1[0],x2[0]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([t,x1[1]+1]))
                if check_belongs_to(l2,u.position):
                    count += 1 
                x1 = udiff.bottom_left
                x2 = udiff.bottom_right
                l1 = list(range(x1[1],x2[1]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([x1[0]+1,t]))
                if check_belongs_to(l2,u.position):
                    count += 1    
                x1 = udiff.top_left
                x2 = udiff.bottom_left
                l1 = list(range(x1[0],x2[0]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([t,x1[1]-1]))
                if check_belongs_to(l2,u.position):
                    count += 1
                if count == 4:
                    regular_non_contractible.append(u)
    return(regular_non_contractible)    

## Filter by size

In [49]:
def filter_by_size(units):
    f = {}
    for j in range(1,7):
        f[j] = []
    for u in units:
        if len(u.position) < 6:
            f[len(u.position)].append(u)
        else:
            f[6].append(u)
    return(f)        

## Assembling image from units

In [50]:
def concat(l,ambient,background = 0):
    r,c = ambient
    a = np.full(ambient,background)
    for x in l:
        for z in x.position:
            if z[0] >= 0 and z[0] <= r-1 and z[1] >= 0 and z[1] <= c-1:
                a[z[0],z[1]] = x.colour
    return(a)      

# Get easier sub-collection of the tasks

In [51]:
def input_output_shape_is_same(task):
    return all([np.array(el['input']).shape == np.array(el['output']).shape for el in task['train']])

def get_equals(task_series):
    l = []
    for j in task_series.index:
        if input_output_shape_is_same(task_series[j]):
            l.append(j)
    return(l) 

In [52]:
l_input_output_same_size = get_equals(train_tasks)
l_input_output_same_size.sort()

In [53]:
# convert a list of np arrays to tuples
def convert_to_tuple(t):
    s = [(a[0],a[1]) for a in t]
    return(s)

In [54]:
# count those images where one must only colour match
l_colour_match_no_diag = []
for index in l_input_output_same_size:
    task = train_tasks[index]
    c = 0
    for example in task['train']:
        units_input = get_units(np.array(example['input']))
        units_output = get_units(np.array(example['output']))
        if len(units_input) == len(units_output):
            sub_count = 0
            for u in units_input:
                s1 = set(convert_to_tuple(u.position))
                for v in units_output:
                    s2 = set(convert_to_tuple(v.position))
                    if s1 == s2:
                        sub_count += 1
            if sub_count == len(units_input):
                c += 1
        if c == len(task['train']):        
            l_colour_match_no_diag.append(index) 

## Lists

In [55]:
# outputs a dict with the following encoding : 
# 0 --> all units
# 1 --> non contractible objects
# 2 --> surrounded_list
# 3 --> contained_in list
# 4 --> squares 
# 5 --> vertical_line (not using for additional features)
# 5.5 --> line
# 6 --> horizontal line (not using for additional features)
# 7 --> horizontal rectangle (not using for additional features)
# 7.5 --> rectangle
# 8 --> vertical rectangle (not using for additional features)
# 9 --> -1 (irregular shape)
# 10 --> regular shapes (union of 14,15,16,17,18) 
# 11 --> regular non contractible units
# 12 --> List of shapes with no doubles
# 13 --> shapes which are doubled
# 14 --> Shapes which are trebled
# 15 --> Shapes that appear in all train inputs (not using for additional features)
# 16 --> Shapes of size 1
# 17 --> Shapes of size 2
# 18 --> Shapes of size 3
# 19 --> Shapes of size 4
# 20 --> Shapes of size 5
# 21 --> Shapes of size greater than 5
# All negative keys are complements of the corresponding positive keys

def get_lists(img,task = None,num = -1,with_diag = False):
    list_dict = {}
    img = np.array(img)
    ambient = np.array(img).shape
    units = get_units(img,with_diag)
    list_dict[0] = units
    #c_dict = get_colour_dict(units)
    #for i in range(10):
    #    list_dict[i+1] = c_dict[i]
    #t = get_up_to_homotopy(units)
    list_dict[1] = get_non_contractible(units,with_diag,False)
    list_dict[2] = surrounded_list(units)
    list_dict[3] = contained_list(units)
    shape_d = get_regular_shapes(units)
    list_dict[4] = shape_d['square']
    #list_dict[5] = shape_d['vertical line']
    #list_dict[6] = shape_d['horizontal line']
    list_dict[5.5] = shape_d['vertical line'] + shape_d['horizontal line']
    #list_dict[7] = shape_d['horizontal rectangle']
    #list_dict[8] = shape_d['vertical rectangle']
    list_dict[7.5] = shape_d['horizontal rectangle'] + shape_d['vertical rectangle']
    list_dict[9] = shape_d['-1']
    list_dict[10] = shape_d['square'] + shape_d['vertical line'] + shape_d['horizontal line'] + shape_d['horizontal rectangle'] + shape_d['vertical rectangle']
    #for count,i in enumerate(range(14,21)):
    #    d_temp = get_colour_dict(list_dict[i])
    #    t = extract_lists_from_dict(d_temp)
    #    for j in range(10):
    #        list_dict[21+(count*10)+j] = t[j]
    list_dict[11] = detect_regular_non_contractible(units)
    shape_counts = shape_equivalences_counts(units,False)
    for i in range(1,4):
        if i not in shape_counts.keys():
            shape_counts[i] = []
    for i in range(1,4):
        list_dict[11+i] = shape_counts[i]
    #list_dict[15] = common_shapes_across_images(img,task,units,num)
    f = filter_by_size(units)
    for j in range(1,7):
        list_dict[15+j] = f[j]
    #list_dict = get_list_dict_complements(units,list_dict) 
    return(units,list_dict)    

In [56]:
# task_number == -1 if test
def additional_X_features(img,task = None,task_number = -1,with_diag = False):
    img = np.array(img)
    units,list_dict = get_lists(img,task,task_number,with_diag)
    d = {}
    r,c = img.shape
    bckg = sort_by_size(units)[-1]
    for i in range(r):
        for j in range(c):
            features = []
            if bckg.colour == 0:
                if check_belongs_to([np.array([i,j])],bckg.position):
                    features.append(1)
                else:
                    features.append(0) 
            else:
                features.append(0)
            for key in list_dict.keys():
                if key > 0:
                    temp = 0
                    if len(list_dict[key]) > 0:
                        t = sort_by_position(list_dict[key],0)
                        for idx,u in enumerate(t):
                            if check_belongs_to([np.array([i,j])],u.position):
                                temp = idx+1
                    features.append(temp)
            d[(i,j)] = features
    return(d)        

## Decision Tree

In [57]:
def getiorc(pair):
    inp = pair["input"]
    return pair["input"],pair["output"],len(inp),len(inp[0])
    
def getAround(i,j,inp,size=1):
    #v = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    r,c = len(inp),len(inp[0])
    v = []
    sc = [0]
    for q in range(size):
        sc.append(q+1)
        sc.append(-(q+1))
    for idx,(x,y) in enumerate(product(sc,sc)):
        ii = (i+x)
        jj = (j+y)
        v.append(-1)
        if((0<= ii < r) and (0<= jj < c)):
            v[idx] = (inp[ii][jj])
    return v

def getDiagonal(i,j,r,c):
    return
        
    
def getX(inp,i,j,size):
    z = []
    n_inp = np.array(inp)
    z.append(i)
    z.append(j)
    r,c = len(inp),len(inp[0])
    for m in range(5):
        z.append(i%(m+1))
        z.append(j%(m+1))
    z.append(i+j)
    z.append(i*j)
#     z.append(i%j)
#     z.append(j%i)
    z.append((i+1)/(j+1))
    z.append((j+1)/(i+1))
    z.append(r)
    z.append(c)
    z.append(len(np.unique(n_inp[i,:])))
    z.append(len(np.unique(n_inp[:,j])))
    arnd = getAround(i,j,inp,size)
    z.append(len(np.unique(arnd)))
    z.extend(arnd)
    return z

def getXy(inp,oup,size):
    x = []
    y = []
    r,c = len(inp),len(inp[0])
    for i in range(r):
        for j in range(c):
            x.append(getX(inp,i,j,size))
            y.append(oup[i][j])
    return x,y
    
def getBkgColor(task_json):
    color_dict = defaultdict(int)
    
    for pair in task_json['train']:
        inp,oup,r,c = getiorc(pair)
        for i in range(r):
            for j in range(c):
                color_dict[inp[i][j]]+=1
    color = -1
    max_count = 0
    for col,cnt in color_dict.items():
        if(cnt > max_count):
            color = col
            max_count = cnt
    return color

## Rough work

In [58]:
#task = train_tasks[l_input_output_same_size[2]]
#img = task['train'][0]['input']

In [59]:
#plot_task(task)

In [60]:
def get_num_colors(inp,oup,bl_cols):
    r,c = len(inp),len(inp[0])
    return 

def replace(inp,uni,perm):
    # uni = '234' perm = ['5','7','9']
    #print(uni,perm)
    r_map = { int(c):int(s) for c,s in zip(uni,perm)}
    r,c = len(inp),len(inp[0])
    rp = np.array(inp).tolist()
    #print(rp)
    for i in range(r):
        for j in range(c):
            if(rp[i][j] in r_map):
                rp[i][j] = r_map[rp[i][j]]
    return rp
            
    
def augment(inp,oup,bl_cols):
    cols = "0123456789"
    npr_map = [1,9,72,3024,15120,60480,181440,362880,362880]
    uni = "".join([str(x) for x in np.unique(inp).tolist()])
    for c in bl_cols:
        cols=cols.replace(str(c),"")
        uni=uni.replace(str(c),"")
        
    exp_size = len(inp)*len(inp[0])*npr_map[len(uni)]
    
    mod = floor(exp_size/120000)
    mod = 1 if mod==0 else mod
    
    #print(exp_size,mod,len(uni))
    result = []
    count = 0
    for comb in combinations(cols,len(uni)):
        for perm in permutations(comb):
            count+=1
            if(count % mod == 0):
                result.append((replace(inp,uni,perm),replace(oup,uni,perm)))
    return result
            
def get_flips(inp,oup):
    result = []
    n_inp = np.array(inp)
    n_oup = np.array(oup)
    result.append((np.fliplr(inp).tolist(),np.fliplr(oup).tolist()))
    result.append((np.rot90(np.fliplr(inp),1).tolist(),np.rot90(np.fliplr(oup),1).tolist()))
    result.append((np.rot90(np.fliplr(inp),2).tolist(),np.rot90(np.fliplr(oup),2).tolist()))
    result.append((np.rot90(np.fliplr(inp),3).tolist(),np.rot90(np.fliplr(oup),3).tolist()))
    result.append((np.flipud(inp).tolist(),np.flipud(oup).tolist()))
    result.append((np.rot90(np.flipud(inp),1).tolist(),np.rot90(np.flipud(oup),1).tolist()))
    result.append((np.rot90(np.flipud(inp),2).tolist(),np.rot90(np.flipud(oup),2).tolist()))
    result.append((np.rot90(np.flipud(inp),3).tolist(),np.rot90(np.flipud(oup),3).tolist()))
    result.append((np.fliplr(np.flipud(inp)).tolist(),np.fliplr(np.flipud(oup)).tolist()))
    result.append((np.flipud(np.fliplr(inp)).tolist(),np.flipud(np.fliplr(oup)).tolist()))
    return result
    
def gettaskxy(task_json,aug,around_size,bl_cols,flip=True):    
    X = []
    Y = []
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        tx,ty = getXy(inp,oup,around_size)
        X.extend(tx)
        Y.extend(ty)
        if(flip):
            for ainp,aoup in get_flips(inp,oup):
                tx,ty = getXy(ainp,aoup,around_size)
                X.extend(tx)
                Y.extend(ty)
                if(aug):
                    augs = augment(ainp,aoup,bl_cols)
                    for ainp,aoup in augs:
                        tx,ty = getXy(ainp,aoup,around_size)
                        X.extend(tx)
                        Y.extend(ty)
        if(aug):
            augs = augment(inp,oup,bl_cols)
            for ainp,aoup in augs:
                tx,ty = getXy(ainp,aoup,around_size)
                X.extend(tx)
                Y.extend(ty)
    return X,Y

def test_predict(task_json,model,size):
    inp = task_json['test'][0]['input']
    eoup = task_json['test'][0]['output']
    r,c = len(inp),len(inp[0])
    oup = predict(inp,model,size)
    return inp,eoup,oup

def predict(inp,model1,model2,model3,size):
    r,c = len(inp),len(inp[0])
    oup = np.zeros([r,c],dtype=int)
    for i in range(r):
        for j in range(c):
            x1 = getX(inp,i,j,size[0])
            x2 = getX(inp,i,j,size[1])
            x3 = getX(inp,i,j,size[2])
            o1 = model1.predict_proba([x1])
            o2 = model2.predict_proba([x2])
            o3 = model3.predict_proba([x3])
            o = (o1+o2+o3)/3
            o = o[0]
            o = np.argmax(o)
            o = 0 if o<0 else o
            oup[i][j]=o
    return oup

def submit_predict(task_json,model1,model2,model3,size,is_training = False):
    pred_map = {}
    idx=0
    for task_num,pair in enumerate(task_json['test']):
        inp = pair["input"]
        oup = predict(inp,model1,model2,model3,size)
        if check_if_colour_match(task_json):
            oup = correct_pred(task_json,oup,task_num)
        pred_map[idx] = oup.tolist()
        idx+=1
        if is_training:
            plot_images([inp,pair['output'],oup])
    return pred_map

def dumb_predict(task_json):
    pred_map = {}
    idx=0
    for pair in task_json['test']:
        inp = pair["input"]
        pred_map[idx] = [[0,0],[0,0]]
        idx+=1
    return pred_map

## Augmentation + model

In [61]:
#gettaskxy(task_json,True,1,[0],flip=True)

In [62]:
def get_loss(model,task_json,size):
    total = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        eoup = predict(inp,model,size)
        total+= np.sum((np.array(oup) != np.array(eoup)))
    return total

def get_test_loss(model,task_json,size):
    total = 0
    for pair in task_json['test']:
        inp,oup=pair["input"],pair["output"]
        eoup = predict(inp,model,size)
        total+= np.sum((np.array(oup) != np.array(eoup)))
    return total

def get_a_size(task_json):
    return 4;

def get_bl_cols(task_json):
    result = []
    bkg_col = getBkgColor(task_json);
    result.append(bkg_col)
    # num_input,input_cnt,num_output,output_cnt
    met_map = {}
    for i in range(10):
        met_map[i] = [0,0,0,0]
        
    total_ex = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        u,uc = np.unique(inp, return_counts=True)
        inp_cnt_map = dict(zip(u,uc))
        u,uc = np.unique(oup, return_counts=True)
        oup_cnt_map = dict(zip(u,uc))
        
        for col,cnt in inp_cnt_map.items():
            met_map[col][0] = met_map[col][0] + 1
            met_map[col][1] = met_map[col][1] + cnt
        for col,cnt in oup_cnt_map.items():
            met_map[col][2] = met_map[col][2] + 1
            met_map[col][3] = met_map[col][3] + cnt
        total_ex+=1
    
    for col,met in met_map.items():
        num_input,input_cnt,num_output,output_cnt = met
        if(num_input == total_ex or num_output == total_ex):
            result.append(col)
        elif(num_input == 0 and num_output > 0):
            result.append(col)
    
    result = np.unique(result).tolist()
    if(len(result) == 10):
        result.append(bkg_col)
    return np.unique(result).tolist()

In [63]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

#def combine_preds(tid,pm1,pm3,pm5,count):
#    result = []
#    for i in range(len(pm1)):
#        tk_s = tid+"_"+str(i)
#        str_pred = flattener(pm1[i])+" "+flattener(pm3[i])+" "+flattener(pm5[i])
#        #print(tk_s,str_pred)
#        result.append([tk_s,str_pred])
#    return result

In [64]:
def combine_preds(tid,pm1,pm3,pm5):
    result = []
    for i in range(len(pm1)):
        tk_s = tid+"_"+str(i)
        str_pred = flattener(pm1[i])+" "+flattener(pm3[i])+" "+flattener(pm5[i])
        #print(tk_s,str_pred)
        result.append([tk_s,str_pred])
    return result

In [65]:
def calk_score(task_test, predict):
    return [int(np.equal(sample['output'], pred).all()) for sample, pred in zip(task_test, predict)]

In [66]:
def get_pred_list(pred_map):
    l = list(pred_map.keys())
    l.sort()
    preds = []
    for i in l:
        preds.append(pred_map[i])
    return(preds)    

In [67]:
def input_output_shape_is_same(task):
    return all([np.array(el['input']).shape == np.array(el['output']).shape for el in task['train']])

def check_if_colour_match(task):
     if input_output_shape_is_same(task):
        c = 0
        for example in task['train']:
            units_input = get_units(np.array(example['input']))
            units_output = get_units(np.array(example['output']))
            if len(units_input) == len(units_output):
                sub_count = 0
                for u in units_input:
                    s1 = set(convert_to_tuple(u.position))
                    for v in units_output:
                        s2 = set(convert_to_tuple(v.position))
                        if s1 == s2:
                            sub_count += 1
                if sub_count == len(units_input):
                    c += 1
                else:
                    return(False)
            else:
                return(False)
        if c == len(task['train']):        
            return(True)
        else:
            return(False)
     else:
        return(False)

In [68]:
def correct_pred(task,p,task_num):
    p = np.array(p)
    img_test = np.array(task['test'][task_num]['input'])
    ambient = np.array(img_test).shape
    units = get_units(img_test)
    for u in units:
        c = [0]*10
        for x in u.position:
            c[p[x[0],x[1]]] += 1
        u.colour = np.argmax(c)
    temp = (concat(units,ambient))
    return(temp)

## Colour and Counting Modulo Q

In [69]:
def Defensive_Copy(A): 
    n = len(A)
    k = len(A[0])
    L = np.zeros((n,k), dtype = int)
    for i in range(n):
        for j in range(k):
            L[i,j] = 0 + A[i][j]
    return L.tolist()

In [70]:
def Create(task, task_id = 0):
    n = len(task['train'])
    Input = [Defensive_Copy(task['train'][i]['input']) for i in range(n)]
    Output = [Defensive_Copy(task['train'][i]['output']) for i in range(n)]
    Input.append(Defensive_Copy(task['test'][task_id]['input']))
    return Input, Output

In [71]:
def Recolor(task):
    Input = task[0]
    Output = task[1]
    Test_Picture = Input[-1]
    Input = Input[:-1]
    N = len(Input)
    
    for x, y in zip(Input, Output):
        if len(x) != len(y) or len(x[0]) != len(y[0]):
            return -1
        
    Best_Dict = -1
    Best_Q1 = -1
    Best_Q2 = -1
    Best_v = -1
    # v ranges from 0 to 3. This gives an extra flexibility of measuring distance from any of the 4 corners
    Pairs = []
    for t in range(15):
        for Q1 in range(1,8):
            for Q2 in range(1,8):
                if Q1+Q2 == t:
                    Pairs.append((Q1,Q2))
                    
    for Q1, Q2 in Pairs:
        for v in range(4):
    
  
            if Best_Dict != -1:
                continue
            possible = True
            Dict = {}
                      
            for x, y in zip(Input, Output):
                n = len(x)
                k = len(x[0])
                for i in range(n):
                    for j in range(k):
                        if v == 0 or v ==2:
                            p1 = i%Q1
                        else:
                            p1 = (n-1-i)%Q1
                        if v == 0 or v ==3:
                            p2 = j%Q2
                        else :
                            p2 = (k-1-j)%Q2
                        color1 = x[i][j]
                        color2 = y[i][j]
                        if color1 != color2:
                            rule = (p1, p2, color1)
                            if rule not in Dict:
                                Dict[rule] = color2
                            elif Dict[rule] != color2:
                                possible = False
            if possible:
                
                # Let's see if we actually solve the problem
                for x, y in zip(Input, Output):
                    n = len(x)
                    k = len(x[0])
                    for i in range(n):
                        for j in range(k):
                            if v == 0 or v ==2:
                                p1 = i%Q1
                            else:
                                p1 = (n-1-i)%Q1
                            if v == 0 or v ==3:
                                p2 = j%Q2
                            else :
                                p2 = (k-1-j)%Q2
                           
                            color1 = x[i][j]
                            rule = (p1,p2,color1)
                            
                            if rule in Dict:
                                color2 = 0 + Dict[rule]
                            else:
                                color2 = 0 + y[i][j]
                            if color2 != y[i][j]:
                                possible = False 
                if possible:
                    Best_Dict = Dict
                    Best_Q1 = Q1
                    Best_Q2 = Q2
                    Best_v = v
                
                
    if Best_Dict == -1:
        return -1 #meaning that we didn't find a rule that works for the traning cases
    
    #Otherwise there is a rule: so let's use it:
    n = len(Test_Picture)
    k = len(Test_Picture[0])
    
    answer = np.zeros((n,k), dtype = int)
   
    for i in range(n):
        for j in range(k):
            if Best_v == 0 or Best_v ==2:
                p1 = i%Best_Q1
            else:
                p1 = (n-1-i)%Best_Q1
            if Best_v == 0 or Best_v ==3:
                p2 = j%Best_Q2
            else :
                p2 = (k-1-j)%Best_Q2
           
            color1 = Test_Picture[i][j]
            rule = (p1, p2, color1)
            if (p1, p2, color1) in Best_Dict:
                answer[i][j] = 0 + Best_Dict[rule]
            else:
                answer[i][j] = 0 + color1
                                    
           
            
    return answer.tolist()

In [72]:
import time 

def inp_oup_dim_same(task_json):
    return all([ len(pair["input"]) == len(pair["output"]) and len(pair["input"][0]) == len(pair["output"][0]) for pair in task_json['train']])
    

solved_task = 0
total_task = 0
task_ids = []
task_preds = []
#total_score_1 = 0
#total_score_2 = 0
#total_score_3 = 0

start = time.time()

for task_path in test_path.glob("*.json"):
    now = time.time()
    if now - start > 31800:
        break
    else:    
        task_json = json.load(open(task_path))
        tk_id = str(task_path).split("/")[-1].split(".")[0]
        print(tk_id)
        if(inp_oup_dim_same(task_json)):
            a_size = get_a_size(task_json)
            bl_cols = get_bl_cols(task_json)
            
            isflip = False
            X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
            if len(X1) < 2000:
                isflip = True
                X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
            X3,Y3 = gettaskxy(task_json,True,3,bl_cols,isflip)
            X5,Y5 = gettaskxy(task_json,True,5,bl_cols,isflip)
            
            model_1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X1, Y1)
            model_3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X3, Y3)
            model_5 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X5, Y5)
            
            n = np.random.choice(3)
            pred_map_1 = submit_predict(task_json,model_1,model_3,model_5,[1,3,5],False)
            pred_map_5 = submit_predict(task_json,model_5,model_5,model_5,[5,5,5],False)
           
        
            n = len(task['train'])
            m11 = len(task['test'])
            count_bad = 0
            pred_map_3 = {}
            for i in range(m11):
                Input = [Defensive_Copy(task['train'][j]['input']) for j in range(n)]
                Output = [Defensive_Copy(task['train'][j]['output']) for j in range(n)]
                Input.append(Defensive_Copy(task['test'][i]['input']))
                solution = Recolor([Input, Output])
                if solution == -1:
                    count_bad += 1
                    pred_map_3[i] = [[0,0],[0,0]]
                else:
                    pred_map_3[i] = solution
            if count_bad == m11:
                pred_map_3 = submit_predict(task_json,model_3,model_3,model_3,[3,3,3],False)
            
        
        
            #pred_map_5 = submit_predict(task_json,model_5,5,False)
            
            #preds_1 = get_pred_list(pred_map_1)
            #preds_2 = get_pred_list(pred_map_3)
            #preds_3 = get_pred_list(pred_map_5)
            #
            #total_score_1 += calk_score(task_json['test'], preds_1)[0]
            #total_score_2 += calk_score(task_json['test'], preds_2)[0]
            #total_score_3 += calk_score(task_json['test'], preds_3)[0]
            
            for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_3,pred_map_5):
                task_ids.append(tks)
                task_preds.append(str_pred)
                #print(tks,str_pred)
            solved_task+=1
            #break
        else:
            pred_map_1 = dumb_predict(task_json)
            pred_map_3 = dumb_predict(task_json)
            pred_map_5 = dumb_predict(task_json)
            
            for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_3,pred_map_5):
                task_ids.append(tks)
                task_preds.append(str_pred)
                #print(tks,str_pred)
            
        total_task+=1

19bb5feb
358ba94e
2753e76c
27a77e38
09c534e7
16b78196
2072aba6
3d31c5b3
34b99a2b
103eff5b
22a4bbc2
40f6cd08
15663ba9
14754a24
136b0064
03560426
184a9768
0607ce86
2546ccf6
15696249
31adaf00
423a55dc
31d5ba1a
27f8ce4f
1e81d6f9
256b0a75
29700607
0e671a1a
2b01abd0
281123b4
2685904e
351d6448
3f23242b
140c817e
0c9aba6e
292dd178
0f63c0b9
3391f8c0
12eac192
15113be4
1a2e2828
2f0c5170
11e1fe23
2c0b0aff
2697da3f
12997ef3
0becf7df
3ed85e70
1d398264
2c737e39
1d0a4b61
0bb8deee
332efdb3
3194b014
18419cfa
17cae0c1
1990f7a8
0934a4d8
3a301edc
05a7bcf2
08573cc6
0b17323b
1da012fc
32e9702f
0c786b71
00576224
25094a63
0692e18c
1c0d0a4b
3979b1a8
20981f0e
3b4c2228
33b52de3
13713586
137f0df0
414297c0
17b80ad2
1acc24af
1c02dbbe
212895b5
12422b43
2a5f8217
3490cc26
070dd51e
0d87d2a6
009d5c81
1e97544e
00dbd492
37d3e8b2
319f2597
195ba7dc
1a6449f1
1c56ad9f
2037f2c7
20818e16
310f3251
21f83797
0a1d4ef5
0a2355a6
3ee1011a


In [73]:
#total_score = 0
#viable_tasks = 0
#solved = []
#for idx,x in (enumerate(l_colour_match_no_diag)):
#    if idx%10 == 1:
#        print(total_score)
#    task = train_tasks[x]
#    #if input_output_shape_is_same(task) and check_colour_match(task):
#    viable_tasks += 1
#    pred = collect_predictions(task)
#    if calk_score(task['test'], pred)[0] > 0:
#        solved.append(idx)
#    total_score += calk_score(task['test'], pred)[0]
#    #else:
#    #        pred = [el['input'] for el in task['test']]

In [74]:
sub_df = pd.DataFrame({"output_id":task_ids,'output':task_preds})
sub_df.to_csv("submission.csv", index=None)